In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

In [2]:
default_config = {
    "semester": 1,
    "local_search": {
        "algorithm": "simulated_annealing",
        "config": {
            "neighborhood": {
                "algorithm": "random_swap",
                "random_swap": {
                    "neighborhood_size": 25
                },
                "random_range_swap": {
                    "neighborhood_size_factor": 0.1,
                    "range_size_factor": 0.1
                },
                "distance_swap": {
                    "distance_percentage": 0.1
                },
                "swap": False
            },
            "simulated_annealing": {
                "initial_temperature": 100,
                "cooling_rate": 0.1,
                "max_iteration": 250,
                "max_time": 30,
                "max_iteration_without_improvement": 50,
            },
            "tabu_search": {
                "tabu_list_size": 50,
                "max_iteration": 1000,
                "max_time": 60,
                "max_iteration_without_improvement": 100,
                "max_time_without_improvement": 5
            }
        }
    },
    "algorithm": {
        "algorithm": "genetic_algorithm",
        "config": {
            "max_iteration": 500,
            "population_size": 500,
            "elitism_size": 2,
            "fitness": {
                "group_assignment_conflict": {
                    "max_threshold": 3,
                    "conflict_penalty": 1
                },
                "assistant_distribution": {
                    "max_group_threshold": 15,
                    "max_shift_threshold": 1,
                    "group_penalty": 1,
                    "shift_penalty": 1
                }
            },
            "operator": {
                "selection": {
                    "roulette_wheel": True,
                    "tournament": False,
                    "elitism": False,
                    "tournament_size": 15
                },
                "crossover": {
                    "single_point": True,
                    "two_point": False,
                    "uniform": False,
                    "crossover_probability": 0.1,
                    "uniform_probability": 0.5
                },
                "mutation": {
                    "swap": True,
                    "shift": False,
                    "random": False,
                    "mutation_probability": 0.1
                },
                "repair": {
                    "time_slot": True
                }
            }
        }
    }
}




In [3]:
from scheduling_algorithm.data_parser import ModuleData
from scheduling_algorithm.algorithms import (
    GeneticAlgorithm,
    GeneticLocalSearch
)
from scheduling_algorithm.utils.solution_generator import SolutionGenerator
from scheduling_algorithm.factory import WeeklyFactory

In [4]:
modules = ModuleData.get_modules_by_semester(1)

In [5]:
# generator = SolutionGenerator.from_data(default_config)
weekly_generator = SolutionGenerator.from_data(default_config)

Loading configuration...
Configuration loaded successfully.
Creating Genetic Algorithm Object from Configuration File
Creating FitnessManager with fitness functions:  [Fitness(name=GroupAssignmentConflictFitness), Fitness(name=AssistantDistributionFitness)]
Configured selection functions:  [Selection(name=RouletteWheelSelection)]
Configuring crossover operator:  [Crossover(name=SinglePointCrossover)]
Configuring mutation operator:  [Mutation(name=SwapMutation)]
Configuring repair operator:  [Repair(name=RepairTimeSlot)]


In [6]:
import cProfile as profile
import pstats
from pstats import SortKey

profiler = profile.Profile()
profiler.enable()
solution = weekly_generator.generate_solution_weekly_test()
profiler.disable()

Generating population for module 1 week 1
Generating population for module 1 week 2
Generating population for module 1 week 3
Generating population for module 1 week 4
Generating population for module 1 week 5
Module 1 week 5 has no population, all the remaining chapter are already assigned on previous weeks
Starting the schedule generation algorithm...


In [7]:
solution

Chromosome(length=288, fitness=116)

In [8]:
stats = pstats.Stats(profiler).sort_stats(SortKey.TIME)
stats.print_stats()

         15449820 function calls (15345510 primitive calls) in 58.372 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   170128   25.836    0.000   25.836    0.000 {method 'acquire' of '_thread.lock' objects}
    96000    7.234    0.000    7.413    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\operator\repair\time_slot_repair.py:22(__call__)
  6915645    7.226    0.000    7.226    0.000 {method 'copy' of 'dict' objects}
    96000    1.969    0.000    9.194    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\structure\chromosome.py:61(<listcomp>)
   100200    1.806    0.000   30.114    0.000 c:\Users\hdiop\mambaforge\envs\jte-lab-timetabling\lib\concurrent\futures\thread.py:161(submit)
   122231    1.433    0.000    1.726    0.000 c:\Users\hdiop\mambaforge\envs\jte-lab-timetabling\lib\threading.py:236(__init__)
    18023    1.288    0.00

In [ ]:
from collections import namedtuple
TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

In [ ]:
for index, gene in enumerate(solution):
    solution.set_time_slot(index, TimeSlot(1, 1, 1))

In [ ]:
available_time_slots = []
not available_time_slots